This notebook tests that required modules can be imported. It also contains a few examples of the functions in `common.py` to initialize the fields, and calculations of the theoretical peaks that will be used in the following notebooks.

# Testing the setup

In [ ]:
import os
# Remove warning when importing Taichi due to old glibc
os.environ["TI_MANYLINUX2014_OK"] = "1"
import time
import numpy
import numba
import numexpr
import taichi
import matplotlib
import gt4py

# Initializing fields

In [ ]:
from common import initialize_field, plot_field, save_result, compare_results

In [ ]:
NX = 128
NY = 128
NZ = 80

### Initial values

We can initialize the 3D fields with a few different patterns: random, horizontal bars, vertical bars and a square.

In [ ]:
field = initialize_field(NX, NY, NZ, mode="random")
plot_field(field)

In [ ]:
field = initialize_field(NX, NY, NZ, mode="random", num_halo=10)
plot_field(field)

In [ ]:
field = initialize_field(NX, NY, NZ, mode="horizontal-bars")
plot_field(field)

In [ ]:
field = initialize_field(NX, NY, NZ, mode="horizontal-bars", num_halo=10)
plot_field(field)

In [ ]:
field = initialize_field(NX, NY, NZ, mode="vertical-bars")
plot_field(field)

In [ ]:
field = initialize_field(NX, NY, NZ, mode="vertical-bars", num_halo=10)
plot_field(field)

In [ ]:
field = initialize_field(NX, NY, NZ, mode="square")
plot_field(field)

### Dimensions order

The underlying order of the dimensions in the NumPy arrays can be changed to any of the 6 possible permutations by using the kwarg `dim_order`. Plotting the field will still work as expected if we use the same kwarg there.

In [ ]:
dim_order = "ZXY"
field = initialize_field(NX, NY, NZ, dim_order=dim_order, mode="horizontal-bars")
plot_field(field, dim_order=dim_order)
field.shape

In [ ]:
dim_order = "YZX"
field = initialize_field(NX, NY, NZ, dim_order=dim_order, mode="horizontal-bars")
plot_field(field, dim_order=dim_order)
field.shape

### C-style (row-major) and Fortran-style (col-major) arrays

Finally, NumPy arrays can be stored row-major or col-major. More details for N-dimensional arrays will be explained in the [lists and NumPy notebook](./1_lists_numpy.ipynb)

In [ ]:
field = initialize_field(NX, NY, NZ, array_order="C")
print(field.flags)
print(field.shape)
print(field.strides)

In [ ]:
field = initialize_field(NX, NY, NZ, array_order="F")
print(field.flags)
print(field.shape)
print(field.strides)

# Benchmarking in Jupyter


### Measuring running times

Since we will be working with Python code all the project, we will run all the tests, examples and benchmarks in Jupyter notebooks. In order to measure benchmark we will use the built-in magic commands `%timeit` and`%%timeit`.

According to [the IPython documentation](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit)

> In cell mode, the statement in the first line is used as setup code (executed but not timed) and the body of the cell is timed. The cell body has access to any variables created in the setup code.

Let's quickly verify this

In [ ]:
%%timeit
1 + 1

In [ ]:
%%timeit
time.sleep(1)  # This line is executed and measured
1 + 1

In [ ]:
%%timeit time.sleep(1)  # This line is executed but not measured
1 + 1

For slow cells (>5s) we will limit the number of repetitions to just 3 using `%%timeit -n1 -r3`

In [ ]:
%%timeit -n1 -r3
time.sleep(3)  # By default, %%timeit would repeat this cell 7 times

The output of the magic command can be saved to a variable using the other built-in magic command `%capture`

In [ ]:
%%capture result
%%timeit -n1 -r3
time.sleep(3)

In [ ]:
print(type(result))
print(result)

However, it is more useful, to store the output as a `TimeitResult`, so that we can access the avergage and standard deviation values as floats.

In [ ]:
%%timeit -n1 -r3 -o
time.sleep(3)

In [ ]:
result = _

In [ ]:
print(type(result))
print(result.average)
print(result.stdev)

### Saving results

In order to automatize saving the benchmark results we have written the function `save_result()`. The first time we call the function, or if we want to overwrite previous results we can pass `overwrite=True` and `header=True`. Later, we can ommit them and new results will be appended at the end of the file.

In [ ]:
save_result(result, "test_3s_sleep", overwrite=True, header=True)

In [ ]:
save_result(result, "test_3s_sleep_bis")

In [ ]:
!cat results.csv

Let's clean the file and just add the header

In [ ]:
save_result(None, overwrite=True, header=True)

In [ ]:
!cat results.csv

### Comparing results

To avoid confusion about expressions like *"A is x times faster than B"*, "B is y times slower than A", or *"A is z % slower than B"*, in all our notebooks we will only use these two expressions to compare benchmarking times: 

A is x times as fast as B, i.e., $B = xA$, and

A is x % faster than B, i.e., $B - A = \dfrac{xA}{100}$

Example: computing a stencil takes 15.3 seconds on a laptop (A) and 6.5 on a desktop (B)

In [ ]:
A = 15.3
B = 6.5

print(f'The desktop is {compare_results(B, A, "faster")} times as fast as the laptop')
print(f'The desktop is {compare_results(B, A, "faster-%")} faster than the laptop')

Example: A = 42 ms and B = 1.07 s

In [ ]:
A = 42e-3
B = 1.07

print(f'A is {compare_results(A, B, "faster")} times as fast as B')
print(f'A is {compare_results(A, B, "faster-%")} faster than B')

In [ ]:
A = 42e-3
B = 58e-3

print(f'A is {compare_results(A, B, "faster")} times as fast as B')
print(f'A is {compare_results(A, B, "faster-%")} faster than B')

# Theoretical peaks

TODO